In [1]:
import pandas as pd
import numpy as np
import json
from rapidfuzz import fuzz
import plotly.express as px

In [2]:
mypath = '/Users/data/Documents/actas/'

with open(mypath + 'results_table-format_fixed.json') as json_file:
    data = json.load(json_file)

In [6]:
df = pd.DataFrame.from_dict(data)
df = df.sort_values(['entity_id', 'page_num'])
df.head()

,entity_id,field_name,start_phrase,end_phrase,confidence,start_answer,end_answer,page_num,answer,context
0,10026,company_name_csf,1775,2223,100.000000,1808,1814,0,ISYS.-,", bajo la siguiente denominación ISYS.- Este p..."
1,10026,society_type_csf,140,1119,100.000000,724,762,0,", SOCIEDAD ANONIMA DE CAPITAL VARIABLE",VEINTIUN MIL QUINIENTOS SETENTA Y OCHO En la H...
7,10026,place_of_grant,140,1119,0.222685,188,235,0,"Ciudad de Cárdenas, Tabasco, República Mexicana",VEINTIUN MIL QUINIENTOS SETENTA Y OCHO En la H...
8,10026,instrument_number,35,874,0.180042,301,303,1,18,- Este permiso quedará sin efectos si dentré d...
9,10026,duration,1319,1389,0.766938,1365,1388,1,99 noventa y nueve años,DURACION.- La duración de la sociedad será de ...


## Podría dejarlo así, pero me conviene reutilizar lo que ya tengo hecho

In [7]:
# cargo el df con los labels (ground truth)
df_labels = pd.read_csv(mypath + "df_labels.csv")
df_labels.head()

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
0,8774,10227,0,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,07/08/2017,5605,acta constitutiva,Nayarit,30.0,Ney González Sánchez,NaN,NaN,NaN,NaN
1,8780,10227,2,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
2,8779,10227,3,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8789,10227,4,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,acta constitutiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8783,10227,6,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion


## Hago el preprocesamiento que charlamos al ground truth
- Expandir `SA DE CV` a `sociedad anonima de capital variable`
- Convertir `8` a `ocho`. Este lo dejo para después. No me convence que el answer puede venir en dígitos.

In [8]:
df_labels['regimen_social'].value_counts()

SA DE CV         29
SC                5
S DE RL DE CV     5
SAS DE CV         5
Name: regimen_social, dtype: int64

In [10]:
df_labels = df_labels.replace(['SA DE CV'],'sociedad anonima de capital variable')
df_labels = df_labels.replace(['SC'],'sociedad civil')
df_labels = df_labels.replace(['S DE RL DE CV'],'sociedad de responsabilidad limitada de capital variable')
df_labels = df_labels.replace(['SAS DE CV'],'sociedad por acciones simplificada de capital variable')

## Mido el match
Para reutilizar lo que ya tengo hecho, armo el df de predicciones con la misma estructura que tenía antes (que no es la que viene cargando el dict directamente)

In [17]:
data[0]

{'entity_id': '10026',
 'field_name': 'company_name_csf',
 'start_phrase': 1775,
 'end_phrase': 2223,
 'confidence': 100,
 'start_answer': 1808,
 'end_answer': 1814,
 'page_num': 0,
 'answer': 'ISYS.-',
 'context': ', bajo la siguiente denominación ISYS.- Este permiso, quedará condicionado a que en los estatutos de la sociedad que se constituya, se inserte la cláusula de exclusión de extranjeros o el convenio previsto en la fracción I del Artículo 27 Constitucional, de conformidad con lo que establecen los artículos 15 de la Ley de Inversión Extranjera y 14 del Reglamento de la Ley de Inversión Extranjera y del Registro Nacional de Inversiones Extranjeras.'}

In [21]:
pred_columns = list(df.field_name.unique())

In [54]:
#df_preds = pd.DataFrame(columns = pred_columns)
#df_preds.head()

# hace falta inicializar las columnas?
df_preds = pd.DataFrame()

In [53]:
data[0]

{'entity_id': '10026',
 'field_name': 'company_name_csf',
 'start_phrase': 1775,
 'end_phrase': 2223,
 'confidence': 100,
 'start_answer': 1808,
 'end_answer': 1814,
 'page_num': 0,
 'answer': 'ISYS.-',
 'context': ', bajo la siguiente denominación ISYS.- Este permiso, quedará condicionado a que en los estatutos de la sociedad que se constituya, se inserte la cláusula de exclusión de extranjeros o el convenio previsto en la fracción I del Artículo 27 Constitucional, de conformidad con lo que establecen los artículos 15 de la Ley de Inversión Extranjera y 14 del Reglamento de la Ley de Inversión Extranjera y del Registro Nacional de Inversiones Extranjeras.'}

In [55]:
i = 0
for d in data:
    
    #for k in d.keys():
    #    df_preds.loc[i, k] = d[k]
    
    df_preds.loc[i, 'entity_id'] = d['entity_id']
    df_preds.loc[i, 'page_num'] = d['page_num']
    df_preds.loc[i, d['field_name']] = d['answer']
        
    i = i+1
            
df_preds = df_preds.sort_values(['entity_id', 'page_num'])

# convierto la página y entity_id a int
df_preds["page_num"] = df_preds["page_num"].astype(int)
df_preds['entity_id'] = df_preds['entity_id'].astype(int)

# pongo id y pagina al ppio:
for c in ['page_num', 'entity_id']:
    first_column = df_preds.pop(c)
    df_preds.insert(0, c, first_column)

### Preprocessing

In [56]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_cols = list(df_preds.select_dtypes(include=numerics))

In [57]:
# convierto todo a lower case
for col in list(df_preds.drop(numeric_cols, axis=1)):
    df_preds[col] = df_preds[col].str.lower()
    
# elimino acentos
for col in list(df_preds.drop(numeric_cols, axis=1)):
    df_preds[col] = df_preds[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    
df_preds.head()

,entity_id,page_num,company_name_csf,society_type_csf,credit_titles,management_acts,domain_acts,notario,num_notaria,place_of_grant,instrument_number,duration,date,apoderado,company_name,society_type
0,10026,0,isys.-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10026,0,NaN,", sociedad anonima de capital variable",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10026,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ciudad de cardenas, tabasco, republica mexicana",NaN,NaN,NaN,NaN,NaN,NaN
8,10026,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN,NaN,NaN
9,10026,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99 noventa y nueve anos,NaN,NaN,NaN,NaN


## Hago el match

In [66]:
preds_match_cols = ['entity_id', 'company_name_csf', 'society_type_csf', 'notario', 'num_notaria', 'place_of_grant', 'instrument_number', 'duration', 'date', 'company_name', 'society_type']
labels_match_cols = ['documento', 'denominacion_social', 'duracion_sociedad', 'fecha_inscripcion', 'fecha_otorgamiento', 'instrumento_publico', 'lugar_otorgamiento', 'notaria_num', 'notario', 'regimen_social']

In [108]:
#df_preds_tmp = df_preds[['entity_id', 'company_name_csf', 'date', 'notario', 'society_type_csf', 'place_of_grant']]
#df_preds_tmp.rename(columns = {'entity_id':'documento', 'notario':'notario_pred', 'date':'date_pred', 'notario':'notario_pred', 'place_of_grant':'state_csf'}, inplace = True)

df_labels_tmp = df_labels[['documento', 'denominacion_social', 'fecha_otorgamiento', 'notario', 'regimen_social', 'lugar_otorgamiento']]
df_labels_tmp.rename(columns = {'denominacion_social':'company_name', 'fecha_otorgamiento':'date', 'regimen_social':'society_type', 'lugar_otorgamiento':'state'}, inplace = True)


/Users/data/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### Cómo arreglo el warning de arriba?

In [72]:
df_preds_tmp.head()

,documento,company_name_csf,date_pred,notario_pred,society_type_csf,state_csf
0,10026,isys.-,NaN,NaN,NaN,NaN
1,10026,NaN,NaN,NaN,", sociedad anonima de capital variable",NaN
7,10026,NaN,NaN,NaN,NaN,"ciudad de cardenas, tabasco, republica mexicana"
8,10026,NaN,NaN,NaN,NaN,NaN
9,10026,NaN,NaN,NaN,NaN,NaN


In [73]:
df_labels_tmp.head()

,documento,company_name,date,notario,society_type,state
0,10227,NaN,07/08/2017,Ney González Sánchez,NaN,Nayarit
1,10227,Administradora 1124,NaN,NaN,sociedad de responsabilidad limitada de capita...,NaN
2,10227,NaN,NaN,NaN,NaN,NaN
3,10227,NaN,NaN,NaN,NaN,NaN
4,10227,NaN,NaN,NaN,NaN,NaN


In [82]:
# cuántos documentos hay en cada df
print('Cantidad de documentos en preds set:', df_preds_tmp[['documento']].nunique()[0])
print('Cantidad de documentos en labels set:', df_labels_tmp[['documento']].nunique()[0])

Cantidad de documentos en preds set: 235
Cantidad de documentos en labels set: 42


### Está todo igual que en el otro nb para hacer el merge

### Falta colapsar las páginas
Lo hacía en ambos df que van al merge

## Este en realidad no es temp, es el preds original

In [128]:
# este df lo lleno juntando todas las páginas
# para eso necesito iterar por documento. Hago una lista de todos los docs que hay

docs = []
for d in data:
    docs.append(d['entity_id'])
    
docs_unique = sorted(list(set(docs)))

In [129]:
df_preds = pd.DataFrame(docs_unique, columns = ['documento'])

In [130]:
for doc in docs_unique:
    for d in data:
        documento = d['entity_id']
        if documento == doc:
            df_preds.loc[df_preds_tmp['documento'] == doc, d['field_name']] =  d['answer']
            
df_preds.head()

,documento,company_name_csf,society_type_csf,credit_titles,management_acts,domain_acts,notario,num_notaria,place_of_grant,instrument_number,duration,date,apoderado,company_name,society_type
0,10026,ISYS.-,", SOCIEDAD ANONIMA DE CAPITAL VARIABLE",títulos de crédito,Actos de administración,actos de dominio,5 the 2,11,"Ciudad de Cárdenas, Tabasco, República Mexicana",18,99 noventa y nueve años,NaN,NaN,NaN,NaN
1,10227,"""ADMINISTRADORA 1124","""SOCIEDAD DE RESPONSABILIDAD LIMITADA DE CAPIT...",NaN,NaN,NaN,NaN,NO,NaN,NaN,NaN,22 DE NOVIEMBRE DE 2012,NaN,NaN,NaN
2,10329,: ISA ADVANCED CONSULTING SERVICES AND RESEARCH,NaN,títulos de crédito,NaN,NaN,NaN,NaN,NaN,NaN,Indefinida,NaN,EMMANUEL SANTANA RENTERIA,NaN,NaN
3,10426,NUTRIESENCE,SOCIEDAD ANÓNIMA DE CAPITAL VARIABLE,NaN,NaN,NaN,JUAN EMILIO LOMELI ACOSTA,7,"Guadalajara, Jalisco","7,253 SIETE",99 noventa y nueve años,NaN,representante legal independientemente del apo...,NaN,NaN
4,10597,"""SERVICIOS INTEGRALES FUTAKE",SOCIEDAD ANONIMA DE CAPITAL VARIABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noventa y nueve años,NaN,"representantes legítimos, tutores, albaceas y ...",NaN,NaN


## Rehago el de las labels compacto también
El snippet de antes no sirve, porque antes las labels las tomaba de un json y ahora de un df que cargué en csv

In [110]:
# collapse rows
df_labels_tmp = df_labels_tmp.fillna('')
df_labels_tmp = df_labels_tmp.groupby('documento').agg(''.join)

df_labels_tmp.head()

,company_name,date,notario,society_type,state
documento,,,,,
10227,Administradora 1124,07/08/2017,Ney González Sánchez,sociedad de responsabilidad limitada de capita...,Nayarit
10329,ISA ADVANCED CONSULTING SERVICES AND RESEARCH,,,sociedad por acciones simplificada de capital ...,
10900,PRO-INT ENGINEERING,08/09/2016,WALDO ARELLANO ESPINDOLA,sociedad anonima de capital variable,PUEBLA
12192,DESARROLLO DE SOFTWARE MONOBITS,21/01/2016,VIRGILIO CESAR VALDES PORTALES,sociedad de responsabilidad limitada de capita...,BAJA CALIFORNIA
12264,CHARVEL Y PEZUELA GROUP,02/10/2012,JOSE RABAGO ORDOÑEZ,sociedad anonima de capital variable,DISTRITO FEDERAL


## A ver si ahora funciona el merge

In [131]:
df_merge = pd.merge(df_preds, df_labels_tmp, on='documento', how='inner')ss

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat